# Homework Assignment 6: Deep Q Learning <a class="tocSkip">

This weeks homework assignment is build a deep q network to defeat the [Lunar Lander](https://gym.openai.com/envs/LunarLander-v2/) environment in OpenAI’s Gym environment. Use this [repository](https://github.com/AndersonJo/dqn-pytorch) as a helpful guide. Train it and test it, if your algorithm successfully learns how to beat the environment, you’ve successfully completed the assignment. Good luck!

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Exercise" data-toc-modified-id="Exercise-1">Exercise</a></span><ul class="toc-item"><li><span><a href="#Global-Parameters" data-toc-modified-id="Global-Parameters-1.1">Global Parameters</a></span></li><li><span><a href="#ReplayMemory-class" data-toc-modified-id="ReplayMemory-class-1.2">ReplayMemory class</a></span></li><li><span><a href="#DQN-class,-our-Deep-Q-Network-class" data-toc-modified-id="DQN-class,-our-Deep-Q-Network-class-1.3">DQN class, our Deep Q Network class</a></span><ul class="toc-item"><li><span><a href="#Network-design" data-toc-modified-id="Network-design-1.3.1">Network design</a></span></li></ul></li><li><span><a href="#LSTMDQN-class,-a-Deep-Q-Network-class-that-uses-LSTM-memory" data-toc-modified-id="LSTMDQN-class,-a-Deep-Q-Network-class-that-uses-LSTM-memory-1.4">LSTMDQN class, a Deep Q Network class that uses LSTM memory</a></span><ul class="toc-item"><li><span><a href="#Network-design" data-toc-modified-id="Network-design-1.4.1">Network design</a></span></li></ul></li><li><span><a href="#Environment-class,-a-gym-game" data-toc-modified-id="Environment-class,-a-gym-game-1.5">Environment class, a gym game</a></span></li><li><span><a href="#Agent-Class" data-toc-modified-id="Agent-Class-1.6">Agent Class</a></span></li><li><span><a href="#Main-function" data-toc-modified-id="Main-function-1.7">Main function</a></span></li><li><span><a href="#Arguments" data-toc-modified-id="Arguments-1.8">Arguments</a></span></li></ul></li><li><span><a href="#Research" data-toc-modified-id="Research-2">Research</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Voice-search-my-browsing-history" data-toc-modified-id="Voice-search-my-browsing-history-2.0.1">Voice search my browsing history</a></span></li></ul></li></ul></li><li><span><a href="#Big-Questions" data-toc-modified-id="Big-Questions-3">Big Questions</a></span></li><li><span><a href="#Resources" data-toc-modified-id="Resources-4">Resources</a></span></li></ul></div>

# Exercise

In [1]:
# imports
import argparse
import copy
import cv2
import glob
import gym
import logging
import math
import os
import pylab
import re
import sys
import torch

# import gym_ple
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch.nn as nn

from collections import deque, namedtuple
from gym.wrappers import Monitor
from random import random, sample
from scipy.misc import toimage
from torch import nn, optim
from torch.autograd import Variable

from torch.nn import functional as F
from torchvision import transforms as T


%matplotlib inline

USE_CPU = False # can be True or False
GPU = 1 # can be 0 or 1
   

## Global Parameters

In [2]:
# Training
BATCH_SIZE = 16

# Replay Memory
REPLAY_MEMORY = 500

# Epsilon
EPSILON_START = 1.0
EPSILON_END = 0.01
EPSILON_DECAY = 5000

# LSTM Memory
LSTM_MEMORY = 128

# ETC Options
TARGET_UPDATE_INTERVAL = 100
CHECKPOINT_INTERVAL = 500
PLAY_INTERVAL = 90 # 90
PLAY_REPEAT = 1000 # 1000
LEARNING_RATE = 0.01



## ReplayMemory class

In [3]:
class ReplayMemory(object):
    
    def __init__(self, capacity=REPLAY_MEMORY):
        self.capacity = capacity
        self.memory = deque(maxlen=self.capacity)
        self.Transition = namedtuple('Transition', ('state', 'action', 'reward', 'next_state'))
        self._available = False
        
    def put(self, state: np.array, action: torch.LongTensor, reward: np.array, next_state: np.array):
        """doc-string goes here"""
        state = torch.FloatTensor(state)
        reward = torch.FloatTensor([reward])
        if next_state is not None:
            next_state = torch.FloatTensor(next_state)
        transition = self.Transition(state=state, action=action, reward=reward, next_state=next_state)
        self.memory.append(transition)
        
    def sample(self, batch_size):
        transitions = sample(self.memory, batch_size)
        return self.Transition(*(zip(*transitions))) # this notation deserves study
        
    def size(self):
        return len(self.memory)
    
    def is_available(self):
        if self._available:
            return True
        
        if len(self.memory) > BATCH_SIZE:
            self._available = True
        return self._available
    
    

## DQN class, our Deep Q Network class

### Network design

1. the data within the model will move through the layers
2. the "forward" method will pull our 'x' through the layers and then apply ReLU functions to keep our values positive
3. $ input (naction) -> conv1(4, 32) -> ReLU -> conv2(32, 64) -> ReLU -> conv3(64, 64) -> ReLU -> affine1(3136, 512) -> affine2(h) -> output $


In [4]:
class DQN(nn.Module):
    def __init__(self, n_action):
        super(DQN, self).__init__()
        self.n_action = n_action
        
        self.conv1 = nn.Conv2d(4, 32, kernel_size=8, stride=4, padding=0) # (In Channel, Out Channel, ...)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=0)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0)

        self.affine1 = nn.Linear(3136, 512)
        self.affine2 = nn.Linear(512, self.n_action)

    def forward(self, x):
        h = F.relu(self.conv1(x))
        h = F.relu(self.conv2(h))
        h = F.relu(self.conv3(h))
        
        h = F.relu(self.affine1(h.view(h.size(0), -1)))
        h = self.affine2(h)
        return h

## LSTMDQN class, a Deep Q Network class that uses LSTM memory

### Network design

1. the data within the model will move through the layers
2. the "forward" method will pull our 'x' through the layers and then apply ReLU functions to keep our values positive
3. $ input (naction) -> conv1(4, 32) -> ReLU -> conv2(32, 64) -> ReLU -> conv3(64, 64) -> ReLU -> affine1(3136, 512) -> affine2(h) -> output $


In [5]:
class LSTMDQN(nn.Module):
    def __init__(self, n_action):
        super(LSTMDQN, self).__init__()
        self.n_action = n_action
        
        self.conv1 = nn.Conv2d(4, 32, kernel_size=8, stride=1, padding=1)  # (In Channel, Out Channel, ...)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        
        self.lstm = nn.LSTM(16, LSTM_MEMORY, 1)  # (Input, Hidden, Num Layers)
        
        self.affine1 = nn.Linear(LSTM_MEMORY * 64, 512)
        self.affine2 = nn.Linear(512, self.n_action)
        
    def forward(self, x, hidden_state, cell_state):
        # CNN
        h = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2, stride=2))
        h = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2, stride=2))
        h = F.relu(F.max_pool2d(self.conv3(x), kernel_size=2, stride=2))
        h = F.relu(F.max_pool2d(self.conv4(x), kernel_size=2, stride=2))
        
        # LSTM
        h = h.view(h.size(0), h.size(1), 16)  # (32, 64, 4, 4) -> (32, 64, 16)
        h, (next_hidden_state, next_cell_state) = self.lstm(h, (hidden_state, cell_state))
        h = h.view(h.size(0), -1) # (32, 64, 256) -> (32, 64 * 256)
        
        # Fully Connected Layers
        h = F.relu(self.affine(h.view(h.size(0) -1)))
        h = self.affine2(h)
        
    def init_states(self) -> [Variable, Variable]:
        hidden_state = Variable(torch.zeros(1, 64, LSTM_MEMORY).cuda())
        cell_state = Variable(torch.zeros(1, 64, LSTM_MEMORY).cuda())
        return hidden_state.detach(), cell_state.detach()

## Environment class, a gym game

In [6]:
class Environment(object):
    def __init__(self, game, record=False, width=84, height=84, seed=0):
        self.game = gym.make(game)
        self.game.seed(seed)
        
        if record:
            self.game = Monitor(self.game, './video', force=True)
            
        self.width = width
        self.height = height
        self._toTensor = T.Compose([T.ToPILImage(), T.ToTensor()])
        # gym_ple
        
    def play_simple(self, mode: str = 'human'):
        observation = self.game.reset()
        
        while True:
            screen = self.game.render(mode=mode)
            if mode == 'rgb_array':
                screen = self.preprocess(screen)
            action = self.game.action_space.sample()
            observation, reward, done, info = self.game.step(action)
            if done:
                break
        self.game.close()
        
    def preprocess(self, screen):
        preprocessed: np.array = cv2.resize(screen, (self.height, self.width)) # 84 * 84
        preprocessed = np.dot(preprocessed[..., :3], [0.299, 0.587, 0.114]) # Gray scale
        preprocessed: np.array = preprocessed.astype('float32') / 255.

        return preprocessed

    def init(self):
        """
        @return observation
        """
        return self.game.reset()

    def get_screen(self):
        screen = self.game.render('rgb_array')
        screen = self.preprocess(screen)
        return screen

    def step(self, action: int):
        observation, reward, done, info = self.game.step(action)
        return observation, reward, done, info

    def reset(self):
        """doc-string"""
        observation = self.game.reset()
        observation = self.preprocess(observation)
        return observation

    @property
    def action_space(self):
        return self.game.action_space.n
        
        
        


## Agent Class

In [11]:
class Agent(object):
    def __init__(self, cuda=True, action_repeat: int = 4):
        """doc-string here"""
        self.clip: bool = CLIP
        self.seed: int = SEED
        self.action_repeat: int = action_repeat
        self.frame_skipping: int = SKIP_ACTION
        self._state_buffer = deque(maxlen=self.action_repeat)
        self.step = 0
        self.best_score = BEST or -10000
        self.best_count = 0

        self._play_steps = deque(maxlen=5)

        # Environment
        self.env = Environment(GAME, record=RECORD, seed=SEED)

        # DQN Model
        self.dqn_hidden_state = self.dqn_cell_state = None
        self.target_hidden_state = self.target_cell_state = None

        self.mode: str = MODEL.lower()
        if self.mode == 'dqn':
            self.dqn: DQN = DQN(self.env.action_space)
        elif self.mode == 'lstm':
            self.dqn: LSTMDQN = LSTMDQN(self.env.action_space)

            # For Optimization
            self.dqn_hidden_state, self.dqn_cell_state = self.dqn.init_states()
            self.target_hidden_state, self.target_cell_state = self.dqn.init_states()

            # For Training Play
            self.train_hidden_state, self.train_cell_state = self.dqn.init_states()

            # For Validation Play
            self.test_hidden_state, self.test_cell_state = self.dqn.init_states()

        if cuda:
            self.dqn.cuda()

        # DQN Target Model
        self.target: DQN = copy.deepcopy(self.dqn)

        # Optimizer
        self.optimizer = optim.Adam(self.dqn.parameters(), lr=LEARNING_RATE)

        # Replay Memory
        self.replay = ReplayMemory()

        # Epsilon
        self.epsilon = EPSILON_START

    def select_action(self, states: np.array) -> tuple:
        """doc-string here"""
        # Decrease epsilon value
        self.epsilon = EPSILON_END + (EPSILON_START - EPSILON_END) * \
            math.exp(-1. * self.step / EPSILON_DECAY)

        if self.epsilon > random():
            # Random Action
            sample_action = self.env.game.action_space.sample()
            action = torch.LongTensor([[sample_action]])
            return action

        states = states.reshape(1, self.action_repeat, self.env.width, self.env.height)
        states_variable: Variable = Variable(torch.FloatTensor(states).cuda())

        if self.mode == 'dqn':
            states_variable.volatile = True
            action = self.dqn(states_variable).data.cpu().max(1)[1].unsqueeze(1)
        elif self.mode == 'lstm':
            action, self.dqn_hidden_state, self.dqn_cell_state = \
                self.dqn(states_variable, self.train_hidden_state, self.train_cell_state)
            action = action.data.cpu().max(1)[1]

        return action

    def get_initial_states(self):
        state = self.env.reset()
        state = self.env.get_screen()
        states = np.stack([state for _ in range(self.action_repeat)], axis=0)

        self._state_buffer = deque(maxlen=self.action_repeat)
        for _ in range(self.action_repeat):
            self._state_buffer.append(state)
        return states

    def add_state(self, state):
        self._state_buffer.append(state)

    def recent_states(self):
        return np.array(self._state_buffer)

    def train(self, gamma: float = 0.99, mode: str = 'rgb_array'):
        # Initial States
        reward_sum = 0.
        q_mean = [0., 0.]
        target_mean = [0., 0.]

        while True:
            # Init LSTM States
            if self.mode == 'lstm':
                # For Training
                self.train_hidden_state, self.train_cell_state = \
                    self.dqn.reset_states(self.train_hidden_state, self.train_cell_state)

            states = self.get_initial_states()
            losses = []
            checkpoint_flag = False
            target_update_flag = False
            play_flag = False
            play_steps = 0
            real_play_count = 0
            real_score = 0

            reward = 0
            done = False
            while True:
                # Get Action
                action: torch.LongTensor = self.select_action(states)

                for _ in range(self.frame_skipping):
                    observation, reward, done, info = self.env.step(int(action))
                    next_state = self.env.get_screen()
                    self.add_state(next_state)

                    if done:
                        break

                # Store the information in Replay Memory
                next_states = self.recent_states()
                if done:
                    self.replay.put(states, action, reward, None)
                else:
                    self.replay.put(states, action, reward, next_states)

                # Change States
                states = next_states

                # Optimize
                if self.replay.is_available():
                    loss, reward_sum, q_mean, target_mean = self.optimize(gamma)
                    losses.append(loss)

                if done:
                    break

                # Increase step
                self.step += 1
                play_steps += 1

                # Target Update
                if self.step % TARGET_UPDATE_INTERVAL == 0:
                    self._target_update()
                    target_update_flag = True

                # Play
                if self.step % PLAY_INTERVAL == 0:
                    play_flag = True

                    scores = []
                    counts = []
                    for _ in range(PLAY_REPEAT):
                        score, real_play_count = self.play(logging=False, human=False)
                        scores.append(score)
                        counts.append(real_play_count)
                        logger.debug(f"[{self.step}] [Validation] play_score: {score}, "
                                     f"play_count: {real_play_count}")

                    real_score = int(np.mean(scores))
                    real_play_count = int(np.mean(counts))

                    if self.best_score <= real_score:
                        self.best_score = real_score
                        self.best_count = real_play_count
                        logger.debug(f"[{self.step}] [Checkpoint] Play: {self.best_score} "
                                     f"[Best Play] [checkpoint]")
                        self.save_checkpoint(
                            filename=f"dqn_checkpoints/chkpoint_{self.mode}_{self.best_score}.pth.tar")

            self._play_steps.append(play_steps)

            # Play
            if play_flag:
                play_flag = False
                logger.info(f"[{self.step}] [Validation] mean_score: {real_score}, mean_play_count: "
                            f"{real_play_count}")

            # Logging
            mean_loss = np.mean(losses)
            target_update_msg = '  [target updated]' if target_update_flag else ''

    def optimize(self, gamma: float):
        if self.mode == "lstm":
            # For Optimization
            self.dqn_hidden_state, self.dqn_cell_state = self.dqn.reset_states(
                self.dqn_hidden_state, self.dqn_cell_state)
            self.target_hidden_state, self.target_cell_state = self.dqn.reset_states(
                self.target_hidden_state, self.target_cell_state)

        # Get Sample
        transitions = self.replay.sample(BATCH_SIZE)

        # Mask
        non_final_mask = \
            torch.ByteTensor(list(map(lambda ns: ns is not None, transitions.next_state))).cuda()

        final_mask = 1 - non_final_mask

        state_batch: Variable = Variable(torch.cat(transitions.state).cuda())
        action_batch: Variable = Variable(torch.cat(transitions.action).cuda())
        reward_batch: Variable = Variable(torch.cat(transitions.reward).cuda())

        next_state_list = [ns for ns in transitions.next_state if ns is not None]
        non_final_next_state_batch = Variable(torch.cat(next_state_list).cuda())
        non_final_next_state_batch.volatile = True

        # Reshape States and Next States
        state_batch = state_batch.view([BATCH_SIZE, self.action_repeat, self.env.width, self.env.height])
        non_final_next_state_batch = non_final_next_state_batch.view(
            [-1, self.action_repeat, self.env.width, self.env.height])
        non_final_next_state_batch.volatile = True
        
        # Predict by DQN Model
        if self.mode == 'dqn':
            q_pred = self.dqn(state_batch)
        elif self.mode == 'lstm':
            q_pred, self.dqn_hidden_state, self.dqn_cell_state = \
                self.dqn(state_batch, self.dqn_hidden_state, self.dqn_cell_state)

        q_values = q_pred.gather(1, action_batch)

        # Predict by Target Model
        target_values = Variable(torch.zeros(BATCH_SIZE, 1).cuda())
        if self.mode == 'dqn':
            target_pred = self.target(non_final_next_state_batch)
        elif self.mode == 'lstm':
            target_pred, self.target_hidden_state, self.target_cell_state = \
                self.target(non_final_next_state_batch, self.target_hidden_state, self.target_cell_state)

        test = reward_batch[non_final_mask] + target_pred.max(1)[0] * gamma
        test = test.unsqueeze(1)

        target_values[non_final_mask] = test
        test2 = reward_batch[final_mask]

        if test2.size()[0] != 0:
            test2 = test2.unsqueeze(1)

        target_values[final_mask] = test2

        loss = F.smooth_l1_loss(q_values, target_values.detach())

        self.optimizer.zero_grad()
        loss.backward()

        if self.clip:
            for param in self.dqn.parameters():
                param.grad.data.clamp_(-1, 1)

        self.optimizer.step()

        reward_score = int(torch.sum(reward_batch).data.cpu().numpy())
        q_mean = torch.sum(q_pred, 0).data.cpu().numpy()[0]
        target_mean = torch.sum(target_pred, 0).data.cpu().numpy()[0]

        return loss.data.cpu().numpy(), reward_score, q_mean, target_mean

    def _target_update(self):
        self.target = copy.deepcopy(self.dqn)

    def save_checkpoint(self, filename='dqn/checkpoints/checkpoint.pth.tar'):
        dirpath = os.path.dirname(filename)

        if not os.path.exists(dirpath):
            os.mkdir(dirpath)

        checkpoint = {
            'dqn': self.dqn.state_dict(),
            'target': self.target.state_dict(),
            'optimizer': self.optimizer.state_dict(),
            'step': self.step,
            'best': self.best_score,
            'best_count': self.best_count
        }
        torch.save(checkpoint, filename)

    def load_checkpoint(self, filename='dqn_checkpoints/checkpoint.pth.tar', epsilon=None):
        checkpoint = torch.load(filename)
        self.dqn.load_state_dict(checkpoint['dqn'])
        self.target.load_state_dict(checkpoint['target'])
        self.optimizer.load_state_dict(checkpoint['optimizer'])
        self.step = checkpoint['step']
        self.best_score = self.best_score or checkpoint['best']
        self.best_count = checkpoint['best_count']

    def load_latest_checkpoint(self, epsilon=None):
        r = re.compile('chkpoint_(dqn|lstm)_(?P<number>-?\d+)\.pth\.tar$')

        files = glob.glob(f'dqn_checkpoints/chkpoint_{self.mode}_*.pth.tar')

        if files:
            files = list(map(lambda x: [int(r.search(x).group('number')), x], files))
            files = sorted(files, key=lambda x: x[0])
            latest_file = files[-1][1]
            self.load_checkpoint(latest_file, epsilon=epsilon)
            print(f'latest checkpoint has been lodaed - {latest_file}')
        else:
            print('no latest checkpoint')

    def play(self, logging=True, human=True):
        self.env.game.close()
        observation = self.env.game.reset()
        states = self.get_initial_states()
        count = 0
        total_score = 0

        self.env.game.seed(self.seed)

        if self.mode == 'lstm':
            self.test_hidden_state, self.test_cell_state = \
                self.dqn.reset_states(self.test_hidden_state, self.test_cell_state)

        while True:
            states = states.reshape(1, self.action_repeat, self.env.width, self.env.height)
            states_variable: Variable = Variable(torch.FloatTensor(states).cuda())

            if self.mode == 'dqn':
                dqn_pred = self.dqn(states_variable)
            elif self.mode == 'lstm':
                dqn_pred, self.dqn_hidden_state, self.dqn_cell_state = \
                    self.dqn(states_variable, self.test_hidden_state, self.test_cell_state)

            action = dqn_pred.data.cpu().max(1)[1][0]
            action = int(action)

            for _ in range(self.frame_skipping):
                if human:
                    screen = self.env.game.render(mode='human')
                observation, reward, done, info = self.env.step(action)
                # States <- Next States
                next_state = self.env.get_screen()
                self.add_state(next_state)
                states = self.recent_states()

                total_score += reward

                if done:
                    break

            # Logging
            count += 1
            if logging:
                action_dist = torch.sum(dqn_pred, 0).data.cpu().numpy()[0]
                print(f'[{count}] action:{action} {action_dist}, reward:{reward}')

            if done:
                break

        self.env.game.close()
        return total_score, count

    def inspect(self):
        print(dir(self.dqn.conv1))

        for param in list(self.dqn.parameters()):
            print(param.size())

        print(self.dqn.conv2.kernel_size)
        print(self.dqn.conv3.kernel_size)
        print(self.dqn.conv4.kernel_size)
        print(self.dqn.conv5.kernel_size)

    @property
    def play_step(self):
        return np.nan_to_num(np.mean(self._play_steps))

    def _sum_params(self, model):
        return np.sum([torch.sum(p).data[0] for p in model.parameters()])

    def imshow(self, sample_image: np.array, transpose=False):
        if transpose:
            sample_image = sample_image.transpose((1, 2, 0))

        pylab.imshow(sample_image, cmap='gray')
        pylab.show()

    def toimage(self, image: np.array, name: str):
        toimage(image, cmin=0, cmax=255).save(name)
                

## Main function

In [12]:
def main():
    agent = Agent()
    if LOAD_LATEST and not CHECKPOINT:
        agent.load_latest_checkpoint()
    elif CHECKPOINT:
        agent.load_checkpoint(CHECKPOINT)
        
    if MODE == 'play':
        agent.play()
    elif MODE == 'train':
        agent.train()
    elif MODE == 'inspect':
        agent.inspect()

        

## Arguments

In [13]:
MODEL = "dqn"        # type=str, default='dqn', help='set model type'
STEP = None          # type=int, default=None, help='forcefully set step'
BEST = None          # type=int, default=None, help='forcefully set best'
LOAD_LATEST = False  # true or false

CHECKPOINT = None        # type=str, default=None
MODE = 'train'           # type=str, one of 'train', 'play', 'inspect'
GAME = "LunarLander-v2"  # type=str
CLIP = True              # clipping the delta between -1 and 1 or not
SKIP_ACTION = 20         # type=int, default=1, help='skipping actions'
RECORD = True            # type=bool
INSPECT = False          # type=bool
SEED = 42                # type=int


torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
np.random.seed(SEED)

logger = logging.getLogger('DQN')
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(message)s')

file_handler = logging.FileHandler(f"dqn_{MODEL}.log")
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)


In [ ]:
main()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


/home/andrew/anaconda3/envs/move_37/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/andrew/anaconda3/envs/move_37/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/andrew/anaconda3/envs/move_37/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/andrew/anaconda3/envs/move_37/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/andrew/anaconda3/envs/move_37/lib/python3.6/site-packages/ipykernel/__main__.py:212: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/andrew/anaconda3/envs/move_37/lib/python3.6/site-packages/ipykernel/__main__.py:218: UserWarning: volatile was removed and no

# Research

In [ ]:
import torch

### Voice search my browsing history
- With a plugin enabled, each site you visit can be searched at a later date using your voice to identify 
- when you visited it, what the page was about, etc.


In [ ]:
torch.cuda.get_device_name(0)

In [ ]:
torch.cuda.get_device_name(1)

In [ ]:
device = torch.device("cuda:1")

In [ ]:
print(device)

In [ ]:
torch.cuda.is_available()

In [ ]:
torch.cuda.device_ctx_manager(0)

---

# Big Questions

1.  Are we copying data from cpu tensors to gpu tensors in our Model class?

# Resources

- [Github 1](https://github.com/etendue/move37/blob/master/dqn_box2d.ipynb)